In [2]:
import sys
print("Python executable:", sys.executable)

import langchain
import langgraph
import sqlalchemy
import pinecone
import sentence_transformers

print("All core imports successful ✅")

Python executable: c:\Users\GCV\dev\work\agentic-gst-ai\gstagent\Scripts\python.exe
All core imports successful ✅
